In [ ]:
days_per_month = {1: 31, 2: 28, 3: 31, 4: 30,
                  5: 31, 6: 30, 7: 31, 8: 31,
                  9: 30, 10: 31, 11: 30, 12: 31}

link_format = "http://www.wunderground.com/history/airport/KNYC/{}/{}/{}/DailyHistory.html"
links = [link_format.format(year, month, day)
         for year in range(2013, 2016) # 2013 - 2015 inclusive
         for month in range(1, 13)     # 1 - 12 inclusive
         for day in range(1, days_per_month[month] + 1)]

print(len(links))
print("\n".join(links[:2]))

1095
http://www.wunderground.com/history/airport/KNYC/2013/1/1/DailyHistory.html
http://www.wunderground.com/history/airport/KNYC/2013/1/2/DailyHistory.html


In [ ]:
import requests
import os.path

def download_file(link, name):
    if os.path.isfile(name):
        return
    file = open(name, 'w')
    r = requests.get(link)
    file.write(r.text)
    file.close()
for i, link in enumerate(links):
    if i % 50 == 0:
        print("Done with %d.." % i)
    download_file(link, "%d.html" % i)

Done with 0..
Done with 50..
Done with 100..
Done with 150..
Done with 200..
Done with 250..


In [ ]:
from bs4 import BeautifulSoup

with open("0.html") as fin:
    soup = BeautifulSoup(fin.read(), "html.parser")

In [ ]:
all_as = soup.find_all('a')
for i in range(5):
    print(all_as[-i])
    print()

In [ ]:
main_table = soup.find(id='historyTable')

In [ ]:
rows = main_table.find_all('tr')
print(len(rows))
for i in range(3):
    print(rows[i])
    print()

In [ ]:
row = rows[2]
for cell in row.find_all('td'):
    print(cell)
    print()
row_name = row.find_all('td')[0].text.strip()  # Get rid of extra whitespace
row_value = row.find_all('td')[1].text.strip()
print(row_name, ":", row_value)


In [ ]:
for row in rows:
    #Only process the rows with 4 cells to eliminate heading rows, etc.
    if len(row.find_all('td')) == 4:
        row_name = row.find_all('td')[0].text.strip()
        row_value = row.find_all('td')[1].text.strip() 
        print(row_name, ":", row_value)    

In [ ]:
fields = ['Mean Temperature', 'Max Temperature', 'Min Temperature',\
          'Dew Point', 'Average Humidity', 'Maximum Humidity',\
          'Minimum Humidity', 'Precipitation', 'Wind Speed',\
          'Max Wind Speed', 'Max Gust Speed']
def scrape_file(name):
    with open(name) as fin:
        soup = BeautifulSoup(fin.read(), "html.parser")
    data = {}
    for row in soup.find(id="historyTable").find_all("tr"):
        cells = row.find_all("td")
        if len(cells) == 4:
            name = cells[0].text.strip()
            if name in fields:
                data[name] = cells[1].text.split()[0].strip()   # Split to remove units
    return data
scrape_file("0.html")

In [ ]:
import pandas as pd

data = pd.read_csv('weather_data.csv')

In [ ]:
pd.options.display.max_rows = 7
data

In [ ]:
ata.columns = [name.lower().replace(" ", "_")
                for name in data.columns]

In [ ]:
data["dew_point"]

In [ ]:
print(data["dew_point"].mean())
data["dew_point"] * 5

In [ ]:
print(len(data))
print(len(data.columns))
data.columns